In [36]:
# Search Engine with Tools and Agents

In [37]:
# Arxiv --> Research papers
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [38]:
# Used the inbuilt tool of arxiv
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [39]:
# Used the inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [40]:
tools = [wiki, arxiv]

In [41]:
# Create Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os

In [42]:
from dotenv import load_dotenv
load_dotenv()

# Load the GROQ API Key
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
os.environ['GEMINI_API_KEY']=os.getenv("GEMINI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [43]:
embeddings = GoogleGenerativeAIEmbeddings(
    google_api_key=os.getenv("GEMINI_API_KEY"),
    model = "models/embedding-001"
)

In [44]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents=documents, embedding=embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022AE2136080>, search_kwargs={})

In [45]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about Langsmith")

retriever_tool.name

'langsmith-search'

In [46]:
tools = [wiki, arxiv, retriever_tool]

In [47]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Rashmi Janve\\OneDrive\\Desktop\\Gen AI Tutorials - Krish Naik\\Langchain Projects\\Chatbots\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000022AE21B0670>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vecto

In [48]:
# Run all the above tools with Agents and LLM Models

# Tools, LLM --> AgentExecutor
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

In [49]:
# Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [52]:
# Agents 
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [54]:
# Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [55]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'what is Langsmith'}`


Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.P

{'input': 'Tell me about Langsmith',
 'output': 'Thank you for providing the tool call result for "call_nhzk". Based on the output, it seems that LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It provides features such as observability, evaluation, and prompt engineering to help developers monitor, evaluate, and optimize their LLM applications.\n\nLangSmith allows developers to closely monitor and evaluate their applications, analyze traces, and configure metrics, dashboards, and alerts. It also provides features for evaluating application performance and getting human feedback on data. Additionally, LangSmith offers prompt engineering tools to help developers find the perfect prompt for their applications.\n\nIt appears that LangSmith is a comprehensive platform for building and deploying LLM applications, and it seems to be a useful tool for developers working with AI and NLP.'}

In [56]:
agent_executor.invoke({"input":"Tell me about Machine Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'Artificial Intelligence'}`


Page: Artificial intelligence
Summary: Artificial intelligence (AI) refers to the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decisio
Invoking: `wikipedia` with `{'query': 'Machine learning algorithms'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witMachin

{'input': 'Tell me about Machine Learning',
 'output': 'Machine learning is a subset of artificial intelligence that involves the use of algorithms to analyze and learn from data. These algorithms can be trained on large datasets and can improve their performance over time, allowing them to make predictions, classify objects, and make decisions.\n\nSome common machine learning algorithms include:\n\n* Supervised learning algorithms, which learn from labeled data and can be used for tasks such as image classification and speech recognition.\n* Unsupervised learning algorithms, which learn from unlabeled data and can be used for tasks such as clustering and anomaly detection.\n* Reinforcement learning algorithms, which learn through trial and error and can be used for tasks such as game playing and robotics.\n\nMachine learning has many applications, including image and speech recognition, natural language processing, recommendation systems, and predictive modeling. It is a rapidly growi

In [57]:
agent_executor.invoke({"input":"Tell me more about this research paper 1706.03762"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cIt seems like the tool call id "call_m8hr" was successful in retrieving information about the research paper with the ID "1706.03762".

Based on the output, it appears to be the paper "Attention Is All You Need" published in 2023. The paper was authored by a group of researchers, and it seems to be a significant work in the field of sequence transduction models.

Now that we have this information, I can respond directly to your original question without needing to use any further tools. The paper "Attention Is All You Need" is a well-known and influential work in the field of natural language processing, and it has been widely cited and built upon. I

{'input': 'Tell me more about this research paper 1706.03762',
 'output': 'It seems like the tool call id "call_m8hr" was successful in retrieving information about the research paper with the ID "1706.03762".\n\nBased on the output, it appears to be the paper "Attention Is All You Need" published in 2023. The paper was authored by a group of researchers, and it seems to be a significant work in the field of sequence transduction models.\n\nNow that we have this information, I can respond directly to your original question without needing to use any further tools. The paper "Attention Is All You Need" is a well-known and influential work in the field of natural language processing, and it has been widely cited and built upon. It presents a new approach to sequence transduction models that relies solely on self-attention mechanisms, rather than the traditional encoder-decoder architecture.\n\nWould you like me to provide more information about this paper, or is there anything else I can